In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
import pickle
import numpy as np
import torch
from facenet_pytorch import InceptionResnetV1
from PIL import Image
import torchvision.transforms as transforms
from imutils import paths

def delete_user_images(username, base_path="userFaces"):
    user_folder = os.path.join(base_path, username)
    
    if not os.path.exists(user_folder):
        messagebox.showerror("Ошибка", f"Пользователь {username} не найден.")
        return
    
    for file in os.listdir(user_folder):
        file_path = os.path.join(user_folder, file)
        os.remove(file_path)  # Удаление файла
    
    os.rmdir(user_folder)  # Удаление пустой папки


def remove_user_from_database(username, database_path="face_enc"):
    if not os.path.exists(database_path):
        return

    with open(database_path, "rb") as f:
        data = pickle.load(f)
    
    new_encodings = []
    new_names = []

    for i in range(len(data["names"])):
        if data["names"][i] != username:
            new_encodings.append(data["encodings"][i])
            new_names.append(data["names"][i])

    # Перезапись файла без удалённого пользователя
    with open(database_path, "wb") as f:
        pickle.dump({"encodings": new_encodings, "names": new_names}, f)
    messagebox.showinfo("Удаление", f"Пользователь {username} удалён из базы.")
    
# Путь к каскаду Хаара
pathHaar = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
detector = cv2.CascadeClassifier(pathHaar)

# Проверка загрузки каскадов Хаара
if detector.empty():
    messagebox.showerror("Ошибка", "Не удалось загрузить каскады Хаара! Проверь путь к файлу.")
    exit()

# Путь к базе данных лиц
database_path = "face_enc"

# Инициализация модели
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = InceptionResnetV1(pretrained="vggface2").eval().to(device)

# Функция для извлечения эмбеддингов лица
def face_encoding(face_image):
    rgb_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb_image)

    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    img_tensor = transform(pil_image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = resnet(img_tensor).squeeze().cpu().numpy()
    
    return embedding

# Функция для вычисления евклидова расстояния
def euclidean_distance(embedding1, embedding2):
    return np.linalg.norm(embedding1 - embedding2)

# Функция для создания папки пользователя
def create_user_folder(base_path, name):
    user_folder = os.path.join(base_path, name)
    os.makedirs(user_folder, exist_ok=True)
    return user_folder

# Получаем пути к изображениям в указанной директории
def get_image_paths(base_dir):
    return list(paths.list_images(base_dir))

# Обрабатываем изображения, извлекаем и сохраняем энкодинги лиц
def process_images(imagePaths):
    knownEncodings = []
    knownNames = []

    # Перебираем все пути к изображениям
    for imagePath in imagePaths:
        # Извлекаем имя пользователя из пути к изображению
        name = os.path.basename(os.path.dirname(imagePath))
        # Загружаем изображение
        image = cv2.imread(imagePath)
        # Конвертируем изображение из BGR в RGB формат
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # encodings = np.array(face_encoding(rgb))
        encodings = np.array(face_encoding(rgb))
        knownEncodings.append(encodings)
        knownNames.append(name)

    
    # Возвращаем словарь с энкодингами и именами
    return {"encodings": knownEncodings, "names": knownNames}

# Сохраняем энкодинги в файл с использованием библиотеки pickle
def save_encodings(data, filename):
    with open(filename, "wb") as f:
        f.write(pickle.dumps(data))

# Функция для захвата лиц с камеры и сохранения их в папку пользователя
def capture_faces(detector, user_folder, name, offset=50, max_images=10):
    # Открываем доступ к камере
    video = cv2.VideoCapture(0)
    i = 0

    while i < max_images:
        # Читаем текущий кадр с камеры
        ret, frame = video.read()
        if not ret:
            break
        
        # Преобразуем кадр в черно-белое изображение
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Обнаруживаем лица на изображении
        faces = detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(200, 200))

        for (x, y, w, h) in faces:
            i += 1
            # Извлекаем область лица с учетом смещения
            face_img = gray[y-offset:y+h+offset, x-offset:x+w+offset]
             # Делаем паузу
            cv2.waitKey(1000)
            # Сохраняем изображение лица в папку пользователя
            cv2.imwrite(f"{user_folder}/face-{name}.{i}.jpg", face_img)
            

     # Получаем пути к изображениям
    imagePaths = get_image_paths('userFaces')
    # Обрабатываем изображения и получаем энкодинги
    data = process_images(imagePaths)
    # Сохраняем энкодинги в файл
    save_encodings(data, "face_enc")
    # Освобождаем камеру
    video.release()
    # Закрываем все окна OpenCV
    cv2.destroyAllWindows()

# Функция для распознавания лица
def recognition():
    if not os.path.exists(database_path):
        messagebox.showerror("Ошибка", "База данных лиц не найдена! Сначала добавьте лица.")
        return
    data = pickle.loads(open('face_enc', "rb").read())

    video_capture = cv2.VideoCapture(0)
    # Цикл обработки кадров видеопотока
    while True:
        # Захватываем кадр из видеопотока
        ret, frame = video_capture.read()
        if not ret:
            break

        # Преобразуем кадр в черно-белое изображение
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(200, 200))
        matches = []
        for (x, y, w, h) in faces:
                # Извлекаем область лица с учетом смещения
                face_img = gray[y-50:y+h+50, x-50:x+w+50]
                faceEncoding = face_encoding(face_img)
               
                for dataEncode in data["encodings"]:
                    euclideDistance = euclidean_distance(dataEncode, faceEncoding)
                    matches.append(euclideDistance < 0.6)

        name = "Unknown"
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b] 
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
                
            name = max(counts, key=counts.get)

        # Выводим сообщения о найденном лице
        if name != "Unknown":
            messagebox.showinfo("Доступ разрешён", f"Добро пожаловать, {name}!")
            video_capture.release()
            cv2.destroyAllWindows()
            break
        else:
            messagebox.showerror("Ошибка доступа", "Доступ запрещён! Лицо не распознано.")
            video_capture.release()
            cv2.destroyAllWindows()

# Пользовательский интерфейс
def fotoFace():
    popup = tk.Toplevel()
    popup.title("Фотография лица")
    popup.geometry("300x150")

    tk.Label(popup, text="Введите имя пользователя:", font=("Arial", 14)).pack(pady=10)
    name_entry = tk.Entry(popup, font=("Arial", 14))
    name_entry.pack(pady=10)

    def start_foto():
        name = name_entry.get()
        if not name:
            messagebox.showerror("Ошибка", "Введите имя пользователя")
            return
        
        pathNewFolder = "userFaces/"
        user_folder = create_user_folder(pathNewFolder, name)
        capture_faces(detector, user_folder, name)
        popup.destroy()
        
    tk.Button(popup, text="Начать захват лиц", command=start_foto).pack(side=tk.LEFT, padx=10)
    tk.Button(popup, text="Закрыть", command=popup.destroy).pack(side=tk.LEFT, padx=10) 

def removeUser():
    popup = tk.Toplevel()
    popup.title("Удаление пользователя")
    popup.geometry("300x150")

    tk.Label(popup, text="Введите имя пользователя:", font=("Arial", 14)).pack(pady=10)
    name_entry = tk.Entry(popup, font=("Arial", 14))
    name_entry.pack(pady=10)

    def remove():
        name = name_entry.get()
        if not name:
            messagebox.showerror("Ошибка", "Введите имя пользователя")
            return
        delete_user_images(name, base_path="userFaces")
        remove_user_from_database(name, database_path="face_enc")
        popup.destroy()
        
    tk.Button(popup, text="Удалить?", command=remove).pack(side=tk.LEFT, padx=10)
    tk.Button(popup, text="Закрыть", command=popup.destroy).pack(side=tk.LEFT, padx=10) 

# Интерфейс Tkinter
def start_interface():
    if detector.empty():
        messagebox.showerror("Ошибка", "Не удалось загрузить каскады Хаара!")
        return
    root = tk.Tk()
    root.title("Распознавание лиц")
    root.geometry("600x300")
    tk.Button(root, text="Добавить пользователя", command=fotoFace, font=("Comic Sans MS", 12)).pack(pady=10)
    tk.Button(root, text="Удалить пользователя", command=removeUser, font=("Comic Sans MS", 12)).pack(pady=10)
    tk.Button(root, text="Получить доступ", command=recognition, font=("Comic Sans MS", 12)).pack(pady=10)
    tk.Button(root, text="Выход", command=root.destroy, font=("Comic Sans MS", 12)).pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    start_interface()


C:\Users\37529\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\37529\AppData\Local\Temp\ipykernel_15332\620769902.py", line 247, in remove
    remove_user_from_database(name, database_path="face_enc")
  File "C:\Users\37529\AppData\Local\Temp\ipykernel_15332\620769902.py", line 45, in remove_user_from_database
    messagebox.info("Удаление", f"Пользователь {username} удалён из базы.")
    ^^^^^^^^^^^^^^^
AttributeError: mod